### I used Cohere.ai's NLP model to...
#### 1. Generate an answer to a silly question (xlarge)
#### 2. Classify labelled restaurant reviews (medium) 
Original open source dataset: https://www.kaggle.com/datasets/d4rklucif3r/restaurant-reviews?select=Restaurant_Reviews.tsv

In [121]:
import pandas as pd
import numpy as np
import cohere
from sklearn.model_selection import train_test_split
pd.set_option('display.max_colwidth', None)

In [122]:
# Install cohere
!pip install cohere

#### Prediction (xlarge)

In [126]:
# Import prediction model
import cohere
co = cohere.Client('o2OB2mK1wit3odwFBcnFHAqKLZ7pG6lCMiYKgtof')
prediction = co.generate(
  model='xlarge',
  prompt='What is the meaning of life?',
#Set max tokens to 50 for a pithy answer! 
  max_tokens=50,
  temperature=1,
  k=0,
  p=0.75)
print('Prediction: {}'.format(prediction.generations[0].text))

Prediction:  The existentialist philosophers say that the purpose of life is to find meaning in life. For them, life is meaningless and the absurd is our everyday existence. To be or not to be? The second wave of existentialists, however, takes a different


##### Very existential!

#### Classify (medium)

In [127]:
df = pd.read_csv('https://raw.githubusercontent.com/carliedeboer/cohere/main/restaurants_reviews.tsv', delimiter='\t', header=None)

#### EDA

In [128]:
df.head()

,0,1
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday off Rick Steve recommendation and loved it.,1
4,The selection on the menu was great and so were the prices.,1


In [129]:
df.count()

0    1000
1    1000
dtype: int64

In [130]:
num_examples = 1000
df_sample = df.sample(num_examples)

#### Train & Test Sets

In [131]:
# Split into train & test sets
sentences_train, sentences_test, labels_train, labels_test = train_test_split(
            list(df_sample[0]), list(df_sample[1]), test_size=0.25, random_state=42)

In [132]:
# Create & retrieve API key
api_key = "o2OB2mK1wit3odwFBcnFHAqKLZ7pG6lCMiYKgtof"
co = cohere.Client(api_key)

In [133]:
# Embed train & test sets
embeddings_train = co.embed(texts=sentences_train,
                             model="large",
                             truncate="LEFT").embeddings

embeddings_test = co.embed(texts=sentences_test,
                             model="large",
                             truncate="LEFT").embeddings

In [134]:
# Import SVM classifier code
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


# Initialize support vector machine, with class_weight='balanced'
svm_classifier = make_pipeline(StandardScaler(), SVC(class_weight='balanced')) 

# Fit the support vector machine
svm_classifier.fit(embeddings_train, labels_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(class_weight='balanced'))])

In [135]:
# Get the score! 
score = svm_classifier.score(embeddings_test, labels_test)
print(f"Validation accuracy on medium is {100*score}%!")

Validation accuracy on medium is 96.39999999999999%!


##### WIN! 